# 预训练语言模型实现与应用

### 案例简介

2018年，Google提出了预训练语言模型BERT，该模型在各种NLP任务上都取得了很好的效果。与此同时，它的使用十分方便，可以快速地对于各种NLP任务进行适配。因此，BERT已经被广泛地使用到了各种NLP任务当中。在本案例中，我们会亲手将BERT适配到长文本关系抽取任务DocRED上，从中了解BERT的基本原理和技术细节。关系抽取是自然语言处理领域的重要任务，DocRED中大部分关系需要从多个句子中联合抽取，因此需要模型具备较强的获取和综合文章中信息的能力，尤其是抽取跨句关系的能力。

### BERT

BERT是目前最具代表性的预训练语言模型，如今预训练语言模型的新方法都是基于BERT进行改进的。研究者如今将各种预训练模型的使用代码整合到了`transformers`这个包当中，使得我们可以很方便快捷地使用各种各样的预训练语言模型。在本实验中，我们也将调用`transformers`来使用BERT完成文档级别关系抽取的任务。

基于`transformers`的基础后，我们的主要工作就是将数据处理成BERT需要的输入格式，以及在BERT的基础上搭建一个能完成特定任务的模型。在本次实验中，我们的重点也将放在这两个方面。

首先是对于数据的处理，对于给定的文本，我们需要使用BERT的tokenizer将文本切成subword，然后转换成对应的id输入进模型中。通常来说这个过程是比较简单的，但是针对于DocRED这个任务，我们需要有一些额外注意的事情。文档级关系抽取的目标是从一段话中确定两个实体之间的关系，为了让模型知道我们关心的两个实体是什么，我们需要在文本中插入四个额外的符号，将实体标注出来。这一部分是数据处理的关键，也是需要同学们自己去动手实现的部分。

与此同时，BERT模型是一个语言模型，为了能使其适配关系抽取任务，我们需要加入额外的神经网络，使得模型能够进行关系预测。通常来说这个神经网络就是将文本中的第一个字符拿出来输入到一个线性层中进行分类。实现一个`BertDocRED`模型是本次作业的第二个任务，

### 数据和代码

本案例使用了DocRED的数据，并提供了一个简单的模型实现，包括数据的预处理、模型的训练、以及简单的评测。中间有两部分代码需要同学进行填充。数据预处理的代码在`preprocessing.ipynb`，里面有一个TODO标识的位置需要进行代码填充。在处理完数据之后，再运行`DocRED.ipynb`进行训练，这里也有一个TODO位置需要进行代码填充。

注意由于预训练模型很大，因此需要调整batch size使得GPU能够放得下，于此同时为了提高batch size的绝对大小，可以使用[梯度累积](https://www.jiqizhixin.com/articles/2018-10-17-11)的技术。我们也提供了对应的实现，具体数值留给同学们进行探索。

### 评分要求

分数由两部分组成。首先，完成对于代码的填空，并且训练模型，评测模型在开发集上的结果，这部分占80%，评分依据为模型的开发集性能和代码的实现情况。第二部分，进行进一步的探索和尝试，我们将在下一小节介绍可能的尝试。同学需要提交代码和报告，在报告中对于两部分的实验都进行介绍，主要包括开发集的结果以及尝试的具体内容。

### 探索和尝试

- 完成对于测试数据的评测，并且提交到DocRED的[评测系统](https://competitions.codalab.org/competitions/20717)中。（推荐先完成该任务，主要考察同学将模型真正应用起来的能力）
- 使用别的预训练语言模型完成该实验，例如RoBERTa等。
- 对于模型进行改进，提升关系抽取的能力，这里可以参考一些DocRED最新工作，进行复现。

### 参考资料

- DocRED: A Large-Scale Document-Level Relation Extraction Dataset. ACL 2019.

In [ ]:
# 一些和环境相关的package

import sys
!{sys.executable} -m pip install torch==1.3.0
!{sys.executable} -m pip install transformers

In [4]:
import random
import os
import argparse
import numpy as np
import json
import torch
import torch.nn as nn
from transformers.configuration_bert import BertConfig
from transformers.optimization import AdamW
from transformers.modeling_bert import BertModel
from torch.utils.data import DataLoader
from tqdm import tqdm

In [5]:
def split_data(features):
    pos_features = []
    neg_features = []
    for feature in features:
        if feature['labels'][0] == 1:
            neg_features.append(feature)
        else:
            pos_features.append(feature)

    return pos_features, neg_features

def combine_shuffle(pos_data, neg_data):
    outputs = []
    for i in range(len(neg_data)):
        outputs.append(torch.cat([pos_data[i], neg_data[i]], 0))
    # shuffle
    rand_index = np.arange(outputs[0].shape[0])
    np.random.shuffle(rand_index)
    for i in range(len(outputs)):
        outputs[i] = outputs[i][rand_index]

    return outputs

def data_gen(data):
    all_input_ids = torch.tensor([f['input_ids'] for f in data], dtype=torch.long)
    all_input_mask = torch.tensor([f['input_mask'] for f in data], dtype=torch.long)
    all_segment_ids = torch.tensor([f['segment_ids'] for f in data], dtype=torch.long)
    all_labels = torch.tensor([f['labels'] for f in data], dtype=torch.float)

    return all_input_ids, all_input_mask, all_segment_ids, all_labels


def get_result(y_pred, y_true,
               acc_num, precision_num, recall_num,
               acc_num_ign, precision_num_ign, recall_num_ign, th=0.5):
    intrain = y_true['intrain']
    y_true = y_true['labels']
    y_pred = y_pred[1:]
    y_true = y_true[1:]
    y_pred[y_pred > th] = 1
    y_pred[y_pred < th] = 0

    y_add = y_pred + y_true
    y_add[y_add != 2] = 0
    y_add[y_add == 2] = 1

    recall_num += np.sum(y_true)
    precision_num += np.sum(y_pred)
    acc_num += np.sum(y_add)

    if not intrain:
        recall_num_ign += np.sum(y_true)
        precision_num_ign += np.sum(y_pred)
        acc_num_ign += np.sum(y_add)

    return acc_num, precision_num, recall_num, acc_num_ign, precision_num_ign, recall_num_ign

def evaluate(model, dev_features, device):
    eval_dataloader = DataLoader(dev_features, batch_size=predict_batch_size, collate_fn=data_gen,
                                 shuffle=False)

    model.eval()
    eval_index = 0
    acc_num = acc_num_ign = 0
    precision_num = precision_num_ign = 0
    recall_num = recall_num_ign = 0

    print("Start evaluating")
    with tqdm(total=len(eval_dataloader), desc='Evaluating') as pbar:
        for batch_data in eval_dataloader:
            input_ids, input_mask, segment_ids, _ = batch_data
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            segment_ids = segment_ids.to(device)

            with torch.no_grad():
                preds = model(input_ids, segment_ids, input_mask)

            preds = preds.cpu().numpy()
            for i in range(preds.shape[0]):
                acc_num, precision_num, recall_num, \
                acc_num_ign, precision_num_ign, recall_num_ign = get_result(preds[i], dev_features[eval_index],
                                                                            acc_num, precision_num, recall_num,
                                                                            acc_num_ign, precision_num_ign,
                                                                            recall_num_ign)
                eval_index += 1

            recall = acc_num / (recall_num + 1e-5)
            precision = acc_num / (precision_num + 1e-5)
            f1 = 2 * (recall * precision) / (recall + precision + 1e-5)

            recall_ign = acc_num_ign / (recall_num_ign + 1e-5)
            precision_ign = acc_num_ign / (precision_num_ign + 1e-5)
            f1_ign = 2 * (recall_ign * precision_ign) / (recall_ign + precision_ign + 1e-5)

            recall *= 100
            precision *= 100
            f1 *= 100
            recall_ign *= 100
            precision_ign *= 100
            f1_ign *= 100

            pbar.set_postfix({'F1': '{:.5f}'.format(f1)})
            pbar.update(1)

    print('Precision:{:.3f}, Recall:{:.3f}, F1-score:{:.3f}'.format(precision, recall, f1))
    print('Precision_ignore:{:.3f}, Recall_ignore:{:.3f}, F1-score_ignore:{:.3f}'.format(precision_ign, recall_ign,
                                                                                         f1_ign))

    model.train()

In [6]:
# 指定GPU
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
# 指定只有预训练模型的类型
model_name = 'bert-base-uncased'
# param
train_feat_dir = 'dataset/train_annotated_cls_data.txt'
dev_feat_dir = 'dataset/dev_cls_data.txt'
label_num = 97
checkpoint_dir ='check_points'
train_batch_size = 2
context_length = 512
num_train_epochs = 10
predict_batch_size = 32
learning_rate = 3e-5
seed = 42
weight_decay_rate = 0.01
warmup_proportion = 0.05
gradient_accumulation_steps = 1 # 梯度累积
output_dir = "check_points/docre-bert"


In [ ]:
class BertDocRED(torch.nn.Module):

    def __init__(self, config, label_num):
        super(BertDocRED, self).__init__()
        self.label_num = label_num
        self.bert = BertModel(config)
        self.decoder = nn.Linear(config.hidden_size, label_num)

    def forward(self, input_ids, token_type_ids, attention_mask, relation_multi_label=None):

        # TODO
        # 实现DocRED模型的训练和预测过程
        # 可以参考普通文本分类的BERT实现，十分类似
        # https://github.com/huggingface/transformers/blob/master/src/transformers/modeling_bert.py 中 BertForSequenceClassification 的 forward 部分实现

In [ ]:
os.makedirs(output_dir, exist_ok=True)

device = torch.device("cuda")
n_gpu = torch.cuda.device_count()
print("device %s n_gpu %d" % (device, n_gpu))
print("device: {} n_gpu: {} ".format(device, n_gpu))

train_batch_size = int(train_batch_size / gradient_accumulation_steps)

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)

# load data
print('loading data...')
train_features = []
with open(train_feat_dir, 'r') as f:
    for line in tqdm(f):
        train_features.append(json.loads(line))
dev_features = []
with open(dev_feat_dir, 'r') as f:
    for line in tqdm(f):
        dev_features.append(json.loads(line))

# 分割多数类和少数类数据
pos_features, neg_features = split_data(train_features)

In [ ]:
print("***** Running training *****")
print("  Num split examples = %d" % len(pos_features) * 2)
print("  Batch size = %d" % (train_batch_size * gradient_accumulation_steps))

# build the models for training and testing/validation
print('######## init model ########')
bert_config = BertConfig.from_pretrained(model_name)
model = BertDocRED(bert_config, label_num=label_num)
model.bert.from_pretrained(model_name)

model.to(device)
if n_gpu > 1:
    model = torch.nn.DataParallel(model)

# Prepare optimizer
param_optimizer = list(model.named_parameters())

no_decay = ['bias', 'gamma', 'beta']
optimizer_parameters = [
    {'params': [p for n, p in param_optimizer if not any([nd in n for nd in no_decay])],
     'weight_decay_rate': weight_decay_rate},
    {'params': [p for n, p in param_optimizer if any([nd in n for nd in no_decay])],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_parameters,
                     lr=learning_rate,
                     weight_decay=weight_decay_rate)

train_pos_dataloader = DataLoader(pos_features,
                                  batch_size=train_batch_size // 2,
                                  collate_fn=data_gen, shuffle=True)
neg_features = neg_features[:len(pos_features) * (len(neg_features) // len(pos_features))]
train_neg_dataloader = DataLoader(neg_features,
                                  batch_size=int(
                                      train_batch_size // 2 * (len(neg_features) // len(pos_features))),
                                  collate_fn=data_gen, shuffle=True)
assert len(train_pos_dataloader) == len(train_neg_dataloader)

print('start training...')
model.train()
global_step = 0
for epoch in range(int(num_train_epochs)):
    total_loss = 0
    show_dict = set()
    for step, (pos_batch, neg_batch) in enumerate(zip(train_pos_dataloader, train_neg_dataloader)):
        pos_batch = tuple(t.to(device) for t in pos_batch)
        neg_batch = tuple(t[0:train_batch_size // 2].to(device) for t in neg_batch)
        batch = combine_shuffle(pos_batch, neg_batch)
        input_ids, input_mask, segment_ids, relation_multi_label = pos_batch
        loss = model(input_ids, segment_ids, input_mask, relation_multi_label)
        total_loss += torch.mean(loss).item()
        if (step + 1) // gradient_accumulation_steps % 20 == 0 \
                and (step + 1) // gradient_accumulation_steps not in show_dict:
            print('Epoch = %d/%d steps = %d ' %
                  (epoch + 1, num_train_epochs,
                   max(1, (step + 1) // gradient_accumulation_steps)) +
                  'loss = %.6f ' % (total_loss / (step + 1)))
            show_dict.add((step + 1) // gradient_accumulation_steps)
        if n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu.

        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
        loss.backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()  # We have accumulated enought gradients
            model.zero_grad()

    evaluate(model, dev_features, device)
    model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
    output_model_file = os.path.join(output_dir, 'checkpoint_' + str(global_step) + '.bin')
    torch.save(model_to_save.state_dict(), output_model_file)